# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

# Introduction: The Business Problem


In this project we will try to find an optimal location for a Coffee Shop. Specifically, this report will be targeted to stakeholders interested in opening a **Bars** in **Sydney**, Australia.

Since there are lots of coffee shops in Sydney we will try to detect **locations that are not already crowded with Bars**. We are also particularly interested in **areas with no bars in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data

1. got the location of the sydney city center
2. got the locations of places within 6km of each other
3. Used four square data to get bars 
4. With the data, I was able to get locations that do not have bars close by

### Neighbourhood

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [10]:
address = 'Martin Place, Sydney'

geolocator = Nominatim(user_agent="sydney_explorer")
location = geolocator.geocode(address)
sydney_latitude = location.latitude
sydney_longitude = location.longitude
print('The geograpical coordinate of Martin Place, sydney are {}, {}.'.format(sydney_latitude, sydney_longitude))

The geograpical coordinate of Martin Place, sydney are -33.8678177, 151.210759.


In [20]:
#pip install shapely
import shapely

#pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Sydney center longitude={}, latitude={}'.format(sydney_longitude, sydney_latitude))
x, y = lonlat_to_xy(sydney_longitude, sydney_latitude)
print('Sydney Center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Sydney center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Sydney center longitude=-33.8678177, latitude=151.210759
Sydney Center UTM X=inf, Y=inf
Sydney center longitude=inf, latitude=inf


C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [14]:
sydney_center_x, sydney_center_y = lonlat_to_xy(sydney_longitude, sydney_latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = sydney_center_x - 6000
x_step = 600
y_min = sydney_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(sydney_center_x, sydney_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning:

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This function is deprec

364 candidate neighborhood centers generated.


In [22]:
# create map of New York using latitude and longitude values
map_sydney = folium.Map(location=[sydney_latitude, sydney_longitude], zoom_start=12)

# add markers to map
for lat, lng, in zip(latitudes, longitudes):
    label = 'latitude {}, longitude {}'.format(latitude, longitude)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    
map_sydney

In [23]:
api_key = ''

In [26]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(api_key, sydney_latitude, sydney_longitude)
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(sydney_latitude, sydney_longitude, addr))

Reverse geocoding check
-----------------------
Address of [-33.8678177, 151.210759] is: Shop 6/3 Martin Pl, Sydney NSW 2000, Australia


In [28]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Australia', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [29]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"1A Cross St, Mosman NSW 2088",-33.836897,151.248631,4.675131e+06,-1.524508e+07,5992.495307
1,"51 Prince Albert St, Mosman NSW 2088",-33.834811,151.243953,4.675731e+06,-1.524508e+07,5840.376700
2,"84B Shadforth St, Mosman NSW 2088",-33.832725,151.239274,4.676331e+06,-1.524508e+07,5747.173218
3,"34 Spencer Rd, Mosman NSW 2088",-33.830638,151.234597,4.676931e+06,-1.524508e+07,5715.767665
4,"289 Military Rd, Cremorne NSW 2090",-33.828552,151.229919,4.677531e+06,-1.524508e+07,5747.173218
5,"12 Illiliwa St, Cremorne NSW 2090",-33.826466,151.225243,4.678131e+06,-1.524508e+07,5840.376700
6,"25 Grafton St, Cremorne NSW 2090",-33.824379,151.220566,4.678731e+06,-1.524508e+07,5992.495307
7,"2 Iluka Rd, Mosman NSW 2088",-33.843408,151.253485,4.674231e+06,-1.524456e+07,5855.766389
8,"34 Kardinia Rd, Mosman NSW 2088",-33.841322,151.248806,4.674831e+06,-1.524456e+07,5604.462508
9,"22 Thompson St, Mosman NSW 2088",-33.839235,151.244127,4.675431e+06,-1.524456e+07,5408.326913


In [30]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood. We would consider the coffee shops only

Initialize Foursquare credentials

In [59]:
client_id = '' # your Foursquare ID
client_secret = '' # your Foursquare Secret
access_token = ''
version = '20180604'
limit = 30
print('Your credentails:')
print('CLIENT_ID: ' + client_id)
print('CLIENT_SECRET:' + client_secret)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [39]:
# Category IDs corresponding to Italian bars were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

nightlife_category = '4d4b7105d754a06376d81259' # 'Root' category for all food-related venues

bar_categories = ['4bf58dd8d48988d116941735',
'52e81612bcbc57f1066b7a0d',
'56aa371ce4b08b9a8d57356c',
'4bf58dd8d48988d117941735',
'52e81612bcbc57f1066b7a0e',
'4bf58dd8d48988d11e941735',
'4bf58dd8d48988d118941735',
'4bf58dd8d48988d1d8941735',
'4bf58dd8d48988d119941735',
'4bf58dd8d48988d1d5941735',
'4bf58dd8d48988d120941735',
'4bf58dd8d48988d11b941735',
'4bf58dd8d48988d11c941735',
'4bf58dd8d48988d1d4941735',
'4bf58dd8d48988d11d941735',
'56aa371be4b08b9a8d57354d',
'4bf58dd8d48988d122941735',
'4bf58dd8d48988d123941735']

def is_bar(categories, specific_filter=None):
    bar_words = ['bar', 'pub', 'garden','speakeasy']
    bar = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in bar_words:
            if r in category_name:
                bar = True
        if 'fast food' in category_name:
            bar = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            bar = True
    return bar, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Sydney', '')
    address = address.replace(', Australia', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [42]:
# Let's now go over our neighborhood locations and get nearby bars; we'll also maintain a dictionary of all found bars and all found bar bars

import pickle

def get_bars(lats, lons):
    bars = {}
    bar_bars = {}
    location_bars = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any bar (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, nightlife_category, client_id, client_secret, radius=350, limit=100)
        area_bars = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_barr = is_bar(venue_categories, specific_filter=bar_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                bar = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_barr, x, y)
                if venue_distance<=300:
                    area_bars.append(bar)
                bars[venue_id] = bar
                if is_barr:
                    bar_bars[venue_id] = bar
        location_bars.append(area_bars)
        print(' .', end='')
    print(' done.')
    return bars, bar_bars, location_bars

# Try to load from local file system in case we did this before
bars = {}
bar_bars = {}
location_bars = []
loaded = False
try:
    with open('bars_350.pkl', 'rb') as f:
        bars = pickle.load(f)
    with open('bar_bars_350.pkl', 'rb') as f:
        bar_bars = pickle.load(f)
    with open('location_bars_350.pkl', 'rb') as f:
        location_bars = pickle.load(f)
    print('bar data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    bars, bar_bars, location_bars = get_bars(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('bars_350.pkl', 'wb') as f:
        pickle.dump(bars, f)
    with open('bar_bars_350.pkl', 'wb') as f:
        pickle.dump(bar_bars, f)
    with open('location_bars_350.pkl', 'wb') as f:
        pickle.dump(location_bars, f)
        

Obtaining venues around candidate locations: .

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':


 . . .

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':


 .

...

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\A

 .

C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\A

 . . done.


C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if sys.path[0] == '':
C:\Users\Chima-Seamfix\A

In [45]:
import numpy as np

print('Total number of Nightlife:', len(bars))
print('Total number of Bars:', len(bar_bars))
print('Percentage of Bars: {:.2f}%'.format(len(bar_bars) / len(bars) * 100))
print('Average number of Bars in neighborhood:', np.array([len(r) for r in location_bars]).mean())

Total number of Nightlife: 609
Total number of Bars: 597
Percentage of Bars: 98.03%
Average number of Bars in neighborhood: 2.1043956043956045


In [46]:
print('List of all Nightlife')
print('-----------------------')
for r in list(bars.values())[:10]:
    print(r)
print('...')
print('Total:', len(bars))

List of all Nightlife
-----------------------
('4b05875ff964a520648e22e3', 'The Buena', -33.83204624280171, 151.24451799817095, "76 Middle Head Rd (at Bradley's Head Rd) NSW 2088", 312, True, 4675850.889977036, -15245437.648955014)
('4cba97a6adcd5481ef4324a2', 'Mosman Bar & Grill', -33.83228720415351, 151.24482633792422, '76 Middle Head Rd., Mosman NSW 2088', 292, True, 4675804.735985664, -15245425.278853701)
('4ec4761c82313b672fe90046', 'Rooftop Bar', -33.832242902079706, 151.2446515693004, '76 Middle Head Rd (Bradleys Head Rd), Mosman NSW 2088', 293, True, 4675825.002050362, -15245421.258797308)
('58537f1f1d21ba69bf580cfa', 'Town & Country Bar', -33.82861, 151.2286, '350 Military Road, Cremorne NSW 2090', 122, True, 4677658.628047481, -15244999.679544317)
('5534c1a0498ee5b605c22eb2', 'Barrel Bar and Dining', -33.82836, 151.22912, '3/362 Military Road, Cremorne NSW', 76, True, 4677622.72068087, -15245057.303696077)
('4b700828f964a52022042de3', "Minsky's Hotel", -33.82875085988138, 151

In [47]:
print('List of Bars')
print('---------------------------')
for r in list(bar_bars.values())[:10]:
    print(r)
print('...')
print('Total:', len(bar_bars))

List of Bars
---------------------------
('4b05875ff964a520648e22e3', 'The Buena', -33.83204624280171, 151.24451799817095, "76 Middle Head Rd (at Bradley's Head Rd) NSW 2088", 312, True, 4675850.889977036, -15245437.648955014)
('4cba97a6adcd5481ef4324a2', 'Mosman Bar & Grill', -33.83228720415351, 151.24482633792422, '76 Middle Head Rd., Mosman NSW 2088', 292, True, 4675804.735985664, -15245425.278853701)
('4ec4761c82313b672fe90046', 'Rooftop Bar', -33.832242902079706, 151.2446515693004, '76 Middle Head Rd (Bradleys Head Rd), Mosman NSW 2088', 293, True, 4675825.002050362, -15245421.258797308)
('58537f1f1d21ba69bf580cfa', 'Town & Country Bar', -33.82861, 151.2286, '350 Military Road, Cremorne NSW 2090', 122, True, 4677658.628047481, -15244999.679544317)
('5534c1a0498ee5b605c22eb2', 'Barrel Bar and Dining', -33.82836, 151.22912, '3/362 Military Road, Cremorne NSW', 76, True, 4677622.72068087, -15245057.303696077)
('4b700828f964a52022042de3', "Minsky's Hotel", -33.82875085988138, 151.2295

In [48]:
print('Bars around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_bars[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Bars around location {}: {}'.format(i+1, names))

Bars around location
---------------------------
Bars around location 101: 
Bars around location 102: 
Bars around location 103: 
Bars around location 104: 
Bars around location 105: 
Bars around location 106: 
Bars around location 107: 
Bars around location 108: 
Bars around location 109: Blues Point Hotel, Delicado Foods, Where The Bars Have No Name
Bars around location 110: 


In [51]:
map_sydny = folium.Map(location=[sydney_latitude, sydney_longitude], zoom_start=13)
folium.Marker([sydney_latitude, sydney_longitude], popup='Martin Place').add_to(map_sydney)
for res in bars.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_bar else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_sydney)
map_sydney


# Methodology

1. Get the data of all the night life in Sydney
2. Identify which that are bars with their distance
3. Focus on areas that someone can open a bar.

# Analysis

In [53]:
location_bars_count = [len(res) for res in location_bars]

df_locations['Bars in area'] = location_bars_count

print('Average number of Bars in every area with radius=300m:', np.array(location_bars_count).mean())

df_locations.head(10)

Average number of Bars in every area with radius=300m: 2.1043956043956045


,Address,Latitude,Longitude,X,Y,Distance from center,Bars in area
0,"1A Cross St, Mosman NSW 2088",-33.836897,151.248631,4.675131e+06,-1.524508e+07,5992.495307,0
1,"51 Prince Albert St, Mosman NSW 2088",-33.834811,151.243953,4.675731e+06,-1.524508e+07,5840.376700,2
2,"84B Shadforth St, Mosman NSW 2088",-33.832725,151.239274,4.676331e+06,-1.524508e+07,5747.173218,0
3,"34 Spencer Rd, Mosman NSW 2088",-33.830638,151.234597,4.676931e+06,-1.524508e+07,5715.767665,0
4,"289 Military Rd, Cremorne NSW 2090",-33.828552,151.229919,4.677531e+06,-1.524508e+07,5747.173218,4
5,"12 Illiliwa St, Cremorne NSW 2090",-33.826466,151.225243,4.678131e+06,-1.524508e+07,5840.376700,0
6,"25 Grafton St, Cremorne NSW 2090",-33.824379,151.220566,4.678731e+06,-1.524508e+07,5992.495307,0
7,"2 Iluka Rd, Mosman NSW 2088",-33.843408,151.253485,4.674231e+06,-1.524456e+07,5855.766389,0
8,"34 Kardinia Rd, Mosman NSW 2088",-33.841322,151.248806,4.674831e+06,-1.524456e+07,5604.462508,0
9,"22 Thompson St, Mosman NSW 2088",-33.839235,151.244127,4.675431e+06,-1.524456e+07,5408.326913,0


get the distance to the nearest bar from every area of the city center

In [56]:
distances_to_bar = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in bar_bars.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_bar.append(min_distance)

df_locations['Distance to Bars'] = distances_to_bar

In [58]:
df_locations.head(15)

,Address,Latitude,Longitude,X,Y,Distance from center,Bars in area,Distance to Bars
0,"1A Cross St, Mosman NSW 2088",-33.836897,151.248631,4.675131e+06,-1.524508e+07,5992.495307,0,758.712165
1,"51 Prince Albert St, Mosman NSW 2088",-33.834811,151.243953,4.675731e+06,-1.524508e+07,5840.376700,2,356.080864
2,"84B Shadforth St, Mosman NSW 2088",-33.832725,151.239274,4.676331e+06,-1.524508e+07,5747.173218,0,600.244177
3,"34 Spencer Rd, Mosman NSW 2088",-33.830638,151.234597,4.676931e+06,-1.524508e+07,5715.767665,0,627.538533
4,"289 Military Rd, Cremorne NSW 2090",-33.828552,151.229919,4.677531e+06,-1.524508e+07,5747.173218,4,51.472671
5,"12 Illiliwa St, Cremorne NSW 2090",-33.826466,151.225243,4.678131e+06,-1.524508e+07,5840.376700,0,402.858041
6,"25 Grafton St, Cremorne NSW 2090",-33.824379,151.220566,4.678731e+06,-1.524508e+07,5992.495307,0,771.598623
7,"2 Iluka Rd, Mosman NSW 2088",-33.843408,151.253485,4.674231e+06,-1.524456e+07,5855.766389,0,1797.465347
8,"34 Kardinia Rd, Mosman NSW 2088",-33.841322,151.248806,4.674831e+06,-1.524456e+07,5604.462508,0,1304.619141
9,"22 Thompson St, Mosman NSW 2088",-33.839235,151.244127,4.675431e+06,-1.524456e+07,5408.326913,0,945.082290
